## 프로젝트 준비를 위한 테이블 생성

In [1]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [2]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    """
    DeclarativeBase 를 상속 받은 Base 라는 하위 클래스를 만들고 시작.
    이 Base 에 Mapping 된 클래스들은 database 에서 단일 테이블임.

    `__tablename__` 을 클래스 레벨의 속성으로 지녀야 함.
    """
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [3]:
Base.metadata.create_all(engine)

2025-04-29 07:03:45,102 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 07:03:45,103 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-04-29 07:03:45,104 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-29 07:03:45,105 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2025-04-29 07:03:45,106 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-29 07:03:45,107 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-04-29 07:03:45,108 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-29 07:03:45,109 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2025-04-29 07:03:45,109 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-29 07:03:45,110 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2025-04-29 07:03:45,111 INFO sqlalchemy.engine.Engine [no key 0.00075s] ()
2025-04-29 07:03:45,113 INFO sqlalchemy.engine.

## SQLAlchemy 에서 Session 이란?

이전 강의에서 Session 을 직접 구현하여 대략적으로 알고 있지만 한번 더 세션에 대해 훑어보는 시간을 가져보도록 합시다. 
SQLAlchemy ORM에서 Session은 애플리케이션과 데이터베이스 간의 모든 **영속성 작업(persistence operations)을 관리하는 주요 인터페이스**입니다. 간단히 말해, 여러분의 파이썬 객체(ORM 매핑된 인스턴스)와 데이터베이스 테이블 간의 상호작용을 책임지는 **"작업 단위"**이자 **"컨텍스트"**라고 생각할 수 있습니다.

## Session 생성법

가장 기본적인 방법으로는 아래와 같은 방법으로 세션을 생성할 수 있습니다. 세션을 생성한 뒤 유저 테이블에 데이터를 삽입해보도록 하겠습니다.

In [7]:
from sqlalchemy.orm import Session

with Session(engine) as sess:
    roach = User(
        name="roach",
        fullname="dev roach",
        addresses=[Address(email_address="roach@sqlalchemy.org")]
    )

    john = User(
        name="john",
        fullname="dev john",
        addresses=[Address(email_address="john@sqlalchemy.org")]
    )

    sess.add_all([roach, john])
    sess.commit()

2025-04-29 07:03:59,309 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 07:03:59,314 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-04-29 07:03:59,315 INFO sqlalchemy.engine.Engine [generated in 0.00014s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('roach', 'dev roach')
2025-04-29 07:03:59,316 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-04-29 07:03:59,317 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('john', 'dev john')
2025-04-29 07:03:59,319 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) RETURNING id
2025-04-29 07:03:59,319 INFO sqlalchemy.engine.Engine [generated in 0.00007s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('roach@sqlalchemy.org', 1)
2025-04-29 07:03:59,320 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) 

## SELECT 조회

이제 다시 저장한 데이터를 조회해보도록 하겠습니다.

In [8]:
from sqlalchemy.orm import Session
from sqlalchemy import select

with Session(engine) as sess:
    stmt = select(User).where(User.name == "roach")
    result = sess.execute(stmt).scalar_one_or_none()
    print(result)

2025-04-29 07:04:03,906 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 07:04:03,909 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-04-29 07:04:03,910 INFO sqlalchemy.engine.Engine [cached since 18.75s ago] ('roach',)
User(id=1, name='roach', fullname='dev roach')
2025-04-29 07:04:03,911 INFO sqlalchemy.engine.Engine ROLLBACK


지난번 Session 직접 구현하기에서 배웠던 것처럼 한번 동일한 쿼리를 두번 날리면 `identity_map` 에서 가져오는지 확인해보도록 할까요? 
지난번 구현에서 배웠듯이 `primary_key` 기반으로 `identity_map` 에 등록되므로 primary_key 기반으로 조회를 해보도록 합시다.

In [13]:
from sqlalchemy.orm import Session
from sqlalchemy import select

with Session(engine) as sess:
    stmt = select(User).where(User.id == 1)
    result = sess.execute(stmt).scalar_one_or_none()
    
    stmt = select(User).where(User.id == 1)
    result = sess.execute(stmt).scalar_one_or_none()

    print(result)

2025-04-29 07:10:24,427 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 07:10:24,429 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2025-04-29 07:10:24,429 INFO sqlalchemy.engine.Engine [cached since 131.2s ago] (1,)
2025-04-29 07:10:24,431 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2025-04-29 07:10:24,431 INFO sqlalchemy.engine.Engine [cached since 131.2s ago] (1,)
User(id=1, name='roach', fullname='dev roach')
2025-04-29 07:10:24,432 INFO sqlalchemy.engine.Engine ROLLBACK


예상과는 다르게 `SELECT` 쿼리가 두번실행된것을 확인할 수 있습니다. 이유가 무엇일까요? 
한번 디버깅을 통해 함께 알아보도록 합시다.

In [20]:
from sqlalchemy.orm import Session
from sqlalchemy import select

with Session(engine) as sess:
    stmt = select(User).where(User.id == 1)
    result = sess.execute(stmt).scalar_one_or_none()

    print(sess.identity_map.all_states()[0].__dict__)
    
    stmt = select(User).where(User.id == 1)
    result = sess.execute(stmt).scalar_one_or_none()

    print(result)

2025-04-29 07:15:35,634 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 07:15:35,636 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2025-04-29 07:15:35,637 INFO sqlalchemy.engine.Engine [cached since 449.1s ago] (1,)
{'key': (<class '__main__.User'>, (1,), None), 'identity_token': None, 'session_id': 17, '_instance_dict': <weakref at 0x7f41ac720b80; to 'sqlalchemy.orm.identity.WeakInstanceDict' at 0x7f41ac8da300>, 'load_options': (), 'load_path': CachingEntityRegistry((<Mapper at 0x7f41ad6ea3c0; User>,)), 'runid': 23}
2025-04-29 07:15:35,639 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2025-04-29 07:15:35,640 INFO sqlalchemy.engine.Engine [cached since 449.1s ago] (1,)
User(id=1, name='roach', fullname='dev roach')
2025-04-29 07:15:35,641 INFO sqlalchemy.engine.Engine ROLLBACK


위의 결과를 확인해보면 첫번째 SELECT 이후 `identity_map` 에 유저가 있음에도, 동일 세션의 두번째 `SELECT` 쿼리가 엔진을 향해 발송되었음을 확인할 수 있습니다. 이는 단순히 [Session.execute](https://docs.sqlalchemy.org/en/20/orm/session_api.html#sqlalchemy.orm.Session.execute) 의 동작이 공식문서에 따르면 쿼리를 실행시키고 실행 결과를 Result 로 리턴하기 때문입니다.

따라서 SELECT 쿼리를 두번 날리지 않기 위해서는 우리는 조금 더 ORM 스러운 방식을 이용해볼수 있습니다.

In [14]:
from sqlalchemy.orm import Session

with Session(engine) as sess:
    roach = sess.get(User, 1)
    print(roach)

    roach = sess.get(User, 1)
    print(roach)

2025-04-29 07:12:32,266 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 07:12:32,268 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2025-04-29 07:12:32,269 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (1,)
User(id=1, name='roach', fullname='dev roach')
User(id=1, name='roach', fullname='dev roach')
2025-04-29 07:12:32,271 INFO sqlalchemy.engine.Engine ROLLBACK


위 코드는 SQLAlchemy 의 [Session.get](https://docs.sqlalchemy.org/en/20/orm/session_api.html#sqlalchemy.orm.Session.get) 을 이용한 코드 입니다. 공식문서에서도 가이드 하듯이 세션의 `identity_map` 에 해당 primary_key 를 가진 객체가 없으면, SELECT 를 날려서 가져오게 됩니다.  

아래 공식문서 글에서는 `만료된(expired)` 내용도 다루고 있으나 현재 단계에서 저희는 다루지 않도록 하겠습니다.


>Session.get() is special in that it provides direct access to the identity map of the Session. If the given primary key identifier is present in the local identity map, the object is returned directly from this collection and no SQL is emitted, unless the object has been marked fully expired. If not present, a SELECT is performed in order to locate the object.

> Session.get() also will perform a check if the object is present in the identity map and marked as expired - a SELECT is emitted to refresh the object as well as to ensure that the row is still present. If not, ObjectDeletedError is raised.

이제 ORM 을 사용할때 만약 `primary_key` 로 조회하고 있다면 `Session.get(Entity, pk)` 메소드를 활용하여 조회해야 한다는 사실을 알게 되었을 것입니다. 실제 현업에서 코드를 작성하다보면 `session` 의 시작지점이 해당 코드 시점에서는 정확히 통제하기가 힘듭니다. 한번 현업에서 우리가 위에서 적었던 SELECT 쿼리를 직접 execute 하는 코드를 적었다고 상상해 봅시다. 하지만 호출한 부분에서 session 을 열었고, 그 `session` 을 함께 이용한다면 어떻게 될까요? 

In [23]:
from sqlalchemy.orm import Session
from sqlalchemy import select

with Session(engine) as sess:
    stmt = select(User).where(User.id == 1)
    result = sess.execute(stmt).scalar_one_or_none()

    print(sess.identity_map.all_states()[0].__dict__)
    
    with sess:
        stmt = select(User).where(User.id == 1)
        result = sess.execute(stmt).scalar_one_or_none()

        print(result)

2025-04-29 07:36:14,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 07:36:14,949 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2025-04-29 07:36:14,949 INFO sqlalchemy.engine.Engine [cached since 1716s ago] (1,)
{'key': (<class '__main__.User'>, (1,), None), 'identity_token': None, 'session_id': 20, '_instance_dict': <weakref at 0x7f4192fc9350; to 'sqlalchemy.orm.identity.WeakInstanceDict' at 0x7f4192e589b0>, 'load_options': (), 'load_path': CachingEntityRegistry((<Mapper at 0x7f41ad6ea3c0; User>,)), 'runid': 26}
2025-04-29 07:36:14,951 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2025-04-29 07:36:14,952 INFO sqlalchemy.engine.Engine [cached since 1716s ago] (1,)
User(id=1, name='roach', fullname='dev roach')
2025-04-29 07:36:14,954 INFO sqlalchemy.engine.Engine ROLLBACK


위와 같이 SELECT 문을 한번 더 실행하게 됩니다. 이게 아무렇지 않은것 같아 보여도 I/O 는 컴퓨터 프로그래밍에서 상당히 비싼 자원이므로 주의해야 합니다. 그렇다면 만약 우리가 `Session.get()` 으로 코드를 작성했다면 어떻게 됬을까요?

In [25]:
from sqlalchemy.orm import Session
from sqlalchemy import select

with Session(engine) as sess:
    user = sess.get(User, 1)
    print(user)
    print(sess.identity_map.all_states()[0].__dict__)
    
    with sess:
        user = sess.get(User, 1)
        print(user)

2025-04-29 07:38:46,933 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 07:38:46,934 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2025-04-29 07:38:46,935 INFO sqlalchemy.engine.Engine [cached since 1610s ago] (1,)
User(id=1, name='roach', fullname='dev roach')
{'key': (<class '__main__.User'>, (1,), None), 'identity_token': None, 'session_id': 22, '_instance_dict': <weakref at 0x7f4192e37ec0; to 'sqlalchemy.orm.identity.WeakInstanceDict' at 0x7f4192e58d70>, 'load_options': (), 'load_path': CachingEntityRegistry((<Mapper at 0x7f41ad6ea3c0; User>,)), 'runid': 29}
{'_dict': {(<class '__main__.User'>, (1,), None): <sqlalchemy.orm.state.InstanceState object at 0x7f4192fc1070>}, '_modified': set(), '_wr': <weakref at 0x7f4192e37ec0; to 'sqlalchemy.orm.identity.WeakInstanceDict' at 0x7f4192e58d70>}
User(id=1, name='roach', f

위와 같이 SELECT 쿼리를 한번만 날리는 것을 확인할 수 있습니다. 이런식으로 최적화는 가능화나 SQLAlchemy 의 `identity_map` 은 엄밀히 말하면 `cache` 기능을 모든 쿼리에 대해 수행한다라고 보기는 어렵습니다. [공식문서]()에도 아래와 같은 내용이 나와있는데요.

> Yeee…no. It’s somewhat used as a cache, in that it implements the identity map pattern, and stores objects keyed to their primary key. However, it doesn’t do any kind of query caching. This means, if you say session.query(Foo).filter_by(name='bar'), even if Foo(name='bar') is right there, in the identity map, the session has no idea about that. It has to issue SQL to the database, get the rows back, and then when it sees the primary key in the row, then it can look in the local identity map and see that the object is already there. It’s only when you say query.get({some primary key}) that the Session doesn’t have to issue a query.

간단하게 설명해보자면 결론적으로 [약한참조](https://docs.python.org/ko/3.13/library/weakref.html) 를 사용하여 `identity_map` 을 유지하기 때문에 우리가 동일 세션내라도 예기치 못한 이벤트로 인해 만약 GC 에 의해 해당 객체가 수거 당했다면 해당 `identity_map` 에는 객체가 존재하지 않을 수도 있기 때문입니다.